### setup login to wandb

In [11]:
# through .env keys
import wandb
import os
from dotenv import load_dotenv
load_dotenv()

api_key  = os.getenv("WANDB_API_KEY_2")
wandb.login(key=api_key)
wandb.require("core")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nhom3ds317 (nhomcs331-beeditor) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING `wandb.require('core')` is redundant as it is now the default behavior.


In [ ]:
# # on notebook
# import wandb
# api_key  = "your_api_key_here"
# wandb.login(key=api_key)
# wandb.require("core")

In [12]:
!python train.py --config ../config/exp.yaml --wandb 1

args Namespace(model='alexnet', config='../config/exp.yaml', wandb=1, project='hutech_mushroom', dataset='hutech-dataset:latest')
HERE config! {'project': 'hutech_mushroom', 'dataset': 'hutech-dataset:latest', 'model': 'convnext_tiny', 'pretrained': True, 'freeze': False, 'resize': 224, 'horizontal_flip': 0.5, 'vertical_flip': 0.5, 'random_rotation': 90, 'random_affine': 0.2, 'random_perspective': 0.2, 'edge_enhance': True, 'auto_aug': 1, 'normalize': 0, 'scheduler': 'cosine', 'is_warmup': False, 'scheduler_warmup': 0.01, 'scheduler_step': 20, 'scheduler_gamma': 0.5, 'es_patience': 200, 'num_classes': 4, 'lr': '1e-5', 'num_epochs': 2, 'batch_size': 4, 'loss': 'cross_entropy', 'is_weight': False, 'optimizer': 'adamw', 'weight_decay': 0.00025, 'momentum': 0.9}
wandb: Currently logged in as: nhom3ds317 (nhomcs331-beeditor) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ creating run (0.3s)
wandb: ⣻ creating run (0.3s)
wandb: Tracking run with wandb version 0